<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/Presos_desde_el_2016_al_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob

# Configuración estética
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

def cargar_datos_anuales(archivo, año):
    """
    Carga datos de un año específico con manejo de encoding y limpieza básica
    """
    try:
        df = pd.read_csv(archivo, sep=';', encoding='latin-1', thousands='.', decimal=',')
        df['AÑO'] = año  # Agregamos columna de año
        return df
    except Exception as e:
        print(f"Error al cargar {archivo}: {str(e)}")
        return None

def analizar_datos(df, año):
    """
    Realiza análisis básicos y genera visualizaciones para un año específico
    """
    print(f"\n=== ANÁLISIS PARA EL AÑO {año} ===\n")

    # Estadísticas descriptivas
    print("Estadísticas descriptivas:")
    print(df[['PRESOSS', 'PRESOCS', 'PRESOPEN']].describe().round(2))

    # Totales
    print("\nTotales:")
    print(df[['PRESOSS', 'PRESOCS', 'PRESOPEN']].sum())

    # Visualizaciones
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f'Distribución de Población Carcelaria - Año {año}')

    sns.histplot(df['PRESOSS'], bins=20, kde=True, ax=axes[0])
    axes[0].set_title('Presos sin Sentencia')

    sns.histplot(df['PRESOCS'], bins=20, kde=True, ax=axes[1])
    axes[1].set_title('Presos con Sentencia')

    sns.histplot(df['PRESOPEN'], bins=20, kde=True, ax=axes[2])
    axes[2].set_title('Presos en Penitenciaría')

    plt.tight_layout()
    plt.show()

    # Correlaciones
    print("\nMatriz de correlación:")
    corr_matrix = df[['PRESOSS', 'PRESOCS', 'PRESOPEN']].corr()
    print(corr_matrix)

    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title(f'Correlaciones - Año {año}')
    plt.show()

def analizar_evolucion_temporal(lista_dfs):
    """
    Analiza la evolución temporal comparando múltiples años
    """
    # Combinar todos los datos en un solo DataFrame
    df_completo = pd.concat(lista_dfs, ignore_index=True)

    # Agrupar por año y calcular totales
    df_evolucion = df_completo.groupby('AÑO')[['PRESOSS', 'PRESOCS', 'PRESOPEN']].sum().reset_index()

    # Visualización de evolución
    plt.figure(figsize=(14, 7))
    df_evolucion_melt = df_evolucion.melt(id_vars='AÑO', var_name='Tipo', value_name='Cantidad')

    sns.lineplot(data=df_evolucion_melt, x='AÑO', y='Cantidad', hue='Tipo',
                 marker='o', markersize=8, linewidth=2.5)

    plt.title('Evolución de la Población Carcelaria por Tipo')
    plt.ylabel('Cantidad de Presos')
    plt.xlabel('Año')
    plt.xticks(df_evolucion['AÑO'].unique())
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Tipo de Preso')
    plt.show()

    # Cálculo de variaciones porcentuales
    df_variacion = df_evolucion.set_index('AÑO').pct_change() * 100
    print("\nVariación porcentual anual:")
    print(df_variacion.round(2))

def main():
    # Diccionario para mapear años a archivos (modificar según tus archivos)
    archivos_por_año = {
        2016: 'microdatos_presos_2016.csv',
        2017: 'microdatos_presos_2017.csv',  # Ejemplo - agregar archivos reales
        2018: 'microdatos_presos_2018.csv'   # Ejemplo - agregar archivos reales
    }

    # Cargar datos para cada año disponible
    datos_anuales = []
    for año, archivo in archivos_por_año.items():
        if os.path.exists(archivo):
            print(f"\nCargando datos para el año {año}...")
            df = cargar_datos_anuales(archivo, año)
            if df is not None:
                datos_anuales.append(df)
                analizar_datos(df, año)
            else:
                print(f"No se pudo cargar datos para {año}")
        else:
            print(f"Archivo no encontrado para el año {año}: {archivo}")

    # Si tenemos datos de múltiples años, hacer análisis temporal
    if len(datos_anuales) > 1:
        analizar_evolucion_temporal(datos_anuales)
    else:
        print("\nNo hay suficientes años para analizar evolución temporal")

if __name__ == "__main__":
    main()